In [10]:
import requests
from base64 import b64encode
import json
import pandas as pd
from datetime import datetime, date, timedelta

import sys
import yaml as yml
import pyodbc as cnn

import re
from dateutil.relativedelta import relativedelta
from tqdm import tqdm, trange
   
apikey = "fz/ocDZn9rHm9ZAupGsDNjoBZGSnHO+8vmU/xWiPPvvFzPE0STJnTbnvu6/FvyOpUhp/TBu0txZ4xZYz6J8iyxkUDkBowVTiDflAvH0qdi5NzWSW+iq8L7j77/Qg77/6Wkj1JvxGkBTOtFOpV9saGw=="
clientId = "33604187fc197c87eb8d2f553935ef8e"

page = 1
pagesize = 1000

# Encode the credentials with base64
username = apikey
password = ''
encoded_credentials = b64encode(bytes(f'{username}:{password}',encoding='ascii')).decode('ascii')

# Use Basic Authorization
auth_header = f'Basic {encoded_credentials}'
print(f'Auth header: {auth_header}')

Auth header: Basic Znovb2NEWm45ckhtOVpBdXBHc0ROam9CWkdTbkhPKzh2bVUveFdpUFB2dkZ6UEUwU1RKblRibnZ1Ni9GdnlPcFVocC9UQnUwdHhaNHhaWXo2SjhpeXhrVURrQm93VlRpRGZsQXZIMHFkaTVOeldTVytpcThMN2o3Ny9RZzc3LzZXa2oxSnZ4R2tCVE90Rk9wVjlzYUd3PT06


In [11]:
def get_datetime_Ndays_ago(n):
    N_days_ago_datetime = datetime.now() - timedelta(days=n)
    return N_days_ago_datetime.strftime("%Y-%m-%d %H:%M")

In [6]:
########## Get the Sent Campaigns #########
# Get the Campaign Id from this API for future campaign -related API
campaign_detail_url = f'https://api.createsend.com/api/v3.2/clients/{clientId}/campaigns.json'
response = requests.get(campaign_detail_url, headers={'Authorization': auth_header } ) 
response_campaign_list = response.json()
df_campaign_list = pd.DataFrame.from_dict(response_campaign_list)
df_campaign_list.head()

# dumps Returns string as output
#json_string = json.dumps(response_campaign_list, indent=2)

,Name,FromName,FromEmail,ReplyTo,SentDate,TotalRecipients,CampaignID,Subject,WebVersionURL,WebVersionTextURL
0,Tin's School Fund Campaign,Tin,hninpwint.tin@student.uts.edu.au,hninpwint.tin@student.uts.edu.au,2020-12-08 16:30:00,5,8fd3ba6bd1dd1d58ce10cd28a12f3795,Tin's School Fund Campaign,http://createsend.com/t/t-2792275A54D1EB4C2540...,http://createsend.com/t/t-2792275A54D1EB4C2540...
1,Tin's Test Campaign for Breakfast,Tin,hninpwint.tin@student.uts.edu.au,hninpwint.tin@student.uts.edu.au,2020-12-07 14:20:00,3,319f7bddfdce8c093e96c39726e62048,Tin's Test Campaign,http://createsend.com/t/t-8398EA282987899E2540...,http://createsend.com/t/t-8398EA282987899E2540...
2,Breakfast for Students Campaign,Tin,hninpwint.tin@student.uts.edu.au,hninpwint.tin@student.uts.edu.au,2020-12-01 17:37:00,1,7fac9427e46cd56cf13fed5c723d7801,Tin's Campaign,http://createsend.com/t/t-B4D1C2432919123A2540...,http://createsend.com/t/t-B4D1C2432919123A2540...


In [13]:
### Filter date should come from config file
active_campaign_day = 90
active_period = get_datetime_Ndays_ago (active_campaign_day)

all_active_campaign = df_campaign_list['SentDate'] > active_period 
campaigns_on_active_period = df_campaign_list.loc[all_active_campaign]

campaigns_on_active_period

,Name,FromName,FromEmail,ReplyTo,SentDate,TotalRecipients,CampaignID,Subject,WebVersionURL,WebVersionTextURL
0,Tin's School Fund Campaign,Tin,hninpwint.tin@student.uts.edu.au,hninpwint.tin@student.uts.edu.au,2020-12-08 16:30:00,5,8fd3ba6bd1dd1d58ce10cd28a12f3795,Tin's School Fund Campaign,http://createsend.com/t/t-2792275A54D1EB4C2540...,http://createsend.com/t/t-2792275A54D1EB4C2540...
1,Tin's Test Campaign for Breakfast,Tin,hninpwint.tin@student.uts.edu.au,hninpwint.tin@student.uts.edu.au,2020-12-07 14:20:00,3,319f7bddfdce8c093e96c39726e62048,Tin's Test Campaign,http://createsend.com/t/t-8398EA282987899E2540...,http://createsend.com/t/t-8398EA282987899E2540...
2,Breakfast for Students Campaign,Tin,hninpwint.tin@student.uts.edu.au,hninpwint.tin@student.uts.edu.au,2020-12-01 17:37:00,1,7fac9427e46cd56cf13fed5c723d7801,Tin's Campaign,http://createsend.com/t/t-B4D1C2432919123A2540...,http://createsend.com/t/t-B4D1C2432919123A2540...


In [16]:
#### Get the Summary for all Campaigns ######
summary_all_campaign_dataset = pd.DataFrame()

for campaign_id in campaigns_on_active_period["CampaignID"]:

    url_campaign_summary = f"https://api.createsend.com/api/v3.2/campaigns/{campaign_id}/summary.json"
    response = requests.get(url_campaign_summary, headers={'Authorization': auth_header }) 
    response_campaign_summary = response.json()
    df_campaign_summary = pd.DataFrame(response_campaign_summary,  index=[0])

    df_campaign_summary["date_update"] = datetime.now()
    df_campaign_summary["campaignId"] = campaign_id
    summary_all_campaign_dataset = summary_all_campaign_dataset.append(df_campaign_summary, ignore_index=True) 
    
summary_all_campaign_dataset

,WebVersionURL,WebVersionTextURL,WorldviewURL,Forwards,Likes,Mentions,Recipients,TotalOpened,Clicks,Unsubscribed,Bounced,UniqueOpened,SpamComplaints,date_update,campaignId
0,http://createsend.com/t/t-2792275A54D1EB4C2540...,http://createsend.com/t/t-2792275A54D1EB4C2540...,http://perso116.createsend.com/reports/wv/t/27...,0,0,0,5,9,0,0,1,3,0,2020-12-15 19:25:00.615795,8fd3ba6bd1dd1d58ce10cd28a12f3795
1,http://createsend.com/t/t-8398EA282987899E2540...,http://createsend.com/t/t-8398EA282987899E2540...,http://perso116.createsend.com/reports/wv/t/83...,0,0,0,3,3,1,0,1,1,0,2020-12-15 19:25:00.830530,319f7bddfdce8c093e96c39726e62048
2,http://createsend.com/t/t-B4D1C2432919123A2540...,http://createsend.com/t/t-B4D1C2432919123A2540...,http://perso116.createsend.com/reports/wv/t/B4...,0,0,0,1,2,1,0,0,1,0,2020-12-15 19:25:01.069109,7fac9427e46cd56cf13fed5c723d7801


In [17]:
def insert_to_db(row, cursor):
    cursor.execute('  INSERT  INTO '
                   '          campaign_status_summary '
                   '          (campaign_id, date_update, forwards, likes, mentions, '
                   '           recipients, total_opened, clicks, unsubscribed, bounced, '
                   '           unique_opened, spam_complaints )'
                   '  VALUES   (?,?,?,?,?, '
                   '            ?,?,?,?,?, '
                   '            ?,?)',
                   row['campaignId'], row['date_update'], row['Forwards'], row['Likes'], row['Mentions'],
                   row['Recipients'], row['TotalOpened'], row['Clicks'], row['Unsubscribed'], row['Bounced'],
                   row['UniqueOpened'], row['SpamComplaints'])

In [18]:
# Open the config file
with open(r'dbconfig.yaml') as file:
    config = yml.full_load(file)

# Establish the connection with database server
conn = cnn.connect('DRIVER='+config['instances']['driver']+
                   ';SERVER='+config['instances']['servername']+
                   ';DATABASE='+config['instances']['database']+
                   ';UID='+config['instances']['username']+
                   ';PWD='+config['instances']['password'])

conn.autocommit = False

#check loading configuration
#METHOD: full/incremental
cfg_load_method = config['loading']['method']

In [19]:
#######  Insert into DB ########
### This should be UPDATE #####
cursor = conn.cursor()

try:
    #if full-load, delete all the records from the table
    if (cfg_load_method == 'full'):
        cursor.execute('DELETE FROM campaign_email_summary')
    
    #insert dataframe to DB
    summary_all_campaign_dataset.apply(insert_to_db, cursor = cursor, axis = 1)
      
    conn.commit()
except:
    conn.rollback()
    print(sys.exc_info())
finally:  
    cursor.close()
    

# Delete ???? move to archive table 
# Insert and Update 